In [70]:
from uuid import uuid4
import json

In [71]:
output_path = "tmp/data-processed-cleaned2/data-processed-shuffled0.jsonl"
shuffled_path = "tmp/data-processed-cleaned2/data-processed-shuffled1.jsonl"
shuffled_path2 = "tmp/data-processed-cleaned2/data-processed-shuffled2.jsonl"

In [72]:
import re

def better_question(question: str) -> str:
    question_num_regex = r"^[Cc](?:a|á|à|ả|ã|ạ|â|ấ|ầ|ẩ|ẫ|ậ|ă|ắ|ằ|ẳ|ẵ|ặ)[Uu](?:|ù|ú|ủ|ũ|ụ|ư|ứ|ừ|ử|ữ|ự)\s*\D+\s*\d+\W*"
    return re.sub(question_num_regex, "", question.strip(), 1)


def better_options(options: list[str]) -> list[str]:
    scan_regex = r"^\s*[A-Ga-g]{1}\W+.+$"
    delete_regex = r"^\s*[A-Ga-g]{1}\W+"
    
    check_list = [0] * (ord('G') - ord('A') + 1)
    results = []
    replaced = 0
    for option in options:
        choice_char = ord(option.strip()[0].upper()) - ord('A')
        if 0 <= choice_char and choice_char < len(check_list) and check_list[choice_char] < 2:
            check_list[choice_char] += 1
        if re.match(scan_regex, option):
            results.append(re.sub(delete_regex, "", option, 1))
            replaced += 1
        else:
            results.append(option)
    if replaced == len(options) and all(check < 2 for check in check_list):
        return results
    else:
        return options

In [73]:
from csv import DictReader

rows = []
last = 0
for i in range(3):
    with open(f"inputs2/part{i}.csv") as csv_file:
        reader = DictReader(csv_file)
        rows.extend({**row, "file": i} for row in reader)
        print("Inserted:", len(rows) - last)
        last = len(rows)

# rows = rows[:300]
mapper = {
    "Hard": "Hard",
    "Easy": "Easy",
    "Moderate": "Medium",
    "Challenging": "Challenging",
    "Medium": "Medium",
    "Very Hard": "Challenging",
}
valid_difficulty = set(["Easy", "Medium", "Hard", "Challenging"])

f = open("errors.jsonl", 'w')

def transform(i, row):
    try:
        id = uuid4().hex
        options = [row[key] for key, value in row.items() if key.startswith("option") and value]
        cleaned_options = better_options(options)
        question = row["question"]
        cleaned_question = better_question(row["question"])
        if cleaned_question != question:
            print(f"{id}: {question} -> {cleaned_question}")
        if options != cleaned_options:
            print(f"{id}: {options} -> {cleaned_options}")

        res = {
            "id": id,
            "difficulty_level": mapper[row["difficultLevel"].strip()],
            "medical_topic": [item.strip() for item in row["medicalTopic"].split(",")],
            "question": cleaned_question,
            "options": cleaned_options,
            "option_map": [i for i in range(len(options))],
            "answer": row["correctOption"][-1],
            "answer_index": ord(row["correctOption"][-1]) - ord("A"),
        }
        assert len(res["question"])
        assert res["difficulty_level"] in valid_difficulty
        assert len(res["medical_topic"])
        assert res["answer_index"] >= 0
        assert res["answer_index"] < len(res["options"])
        return res
    except Exception as e:
        json.dump(row, f, ensure_ascii=False)
        f.write('\n')
        # print(i, row)
        # print(e)
        # raise e
        return None

rows = [
    transform(i, row)
    for i, row in enumerate(rows)
]

print(rows[0])
rows = list(filter(lambda row: row is not None, rows))
print("Final len:", len(rows))
f.close()

Inserted: 1924
Inserted: 1999
Inserted: 1416
2bf07cb12baa41bebe61750a3fee94b3: ['A. Morphin và Cocain', 'B. Morphinvà Heroin', 'C. Heroin và Cocain', 'D. Morphin và Codein'] -> ['Morphin và Cocain', 'Morphinvà Heroin', 'Heroin và Cocain', 'Morphin và Codein']
d46d8bf93e3c4fe9bb4bf1c1698c3d53: ['A. Amphetamin', 'B. Aconitin', 'C.Quinine', 'D. Cocain'] -> ['Amphetamin', 'Aconitin', 'Quinine', 'Cocain']
39cc4675d3ec4860ae6ba237a9e8e4fb: ['A. Hoa trắng, hạt màu vàng nhạt', 'B.\nHoa tím, hạt màu xám', 'C. Hoa tím, hạt đen tím', 'D. Hoa hồng, hạt màu tím'] -> ['Hoa trắng, hạt màu vàng nhạt', 'Hoa tím, hạt màu xám', 'Hoa tím, hạt đen tím', 'Hoa hồng, hạt màu tím']
4fafb4608e8b462ba1eb34ae76cccaa3: ['a. Màng căng đàn hồi chịu tác dụng của một lực mạnh.', 'b. Dây kim loại căng chịu tác dụng của một lực tuần hoàn.', 'c. Một bản thạch anh áp điện chịu tác dụng của một hiệu điện thế xoay chiều tần số cao hơn 20.000Hz.', 'd. Cột không khí bị nén bởi một lực nào đó.'] -> ['Màng căng đàn hồi chịu tác

In [74]:
len(rows)

5304

In [75]:
rows[0]

{'id': 'd29c07eb35614948bd3bda4f04bbfe44',
 'difficulty_level': 'Easy',
 'medical_topic': ['Toxicology', 'Emergency Medicine'],
 'question': '_________ từ cá nóc gây tê liệt, sưng tấy và buồn nôn.',
 'options': ['thuốc độc', 'bị đầu độc', 'độc hại', 'ngộ độc'],
 'option_map': [0, 1, 2, 3],
 'answer': 'A',
 'answer_index': 0}

In [76]:
with open(output_path, 'w') as f:
    for row in rows:
        json.dump(row, f, ensure_ascii=False)
        f.write('\n')

In [77]:
import random

def shuffle_options(row):
    try:
        new_option_map: list[int] = row["option_map"].copy()
        random.shuffle(new_option_map)
        # 2 3 1 0
        # 0 1 2 3
        # 2

        # Build new row with shuffled options and updated answer_index
        new_row = row.copy()
        new_row["options"] = [row["options"][i] for i in new_option_map]
        new_row["option_map"] = new_option_map
        new_row["answer_index"] = new_option_map.index(row["answer_index"])
        new_row["answer"] = chr(new_row["answer_index"] + ord('A'))
        return new_row
    except Exception as e:
        print(row)
        print(len(row["options"]))
        print(e)

In [78]:
shuffled_rows = [shuffle_options(row) for row in rows]
shuffled_rows = list(filter(lambda row: row is not None, shuffled_rows))
with open(shuffled_path, 'w') as f:
    for row in shuffled_rows:
        json.dump(row, f, ensure_ascii=False)
        f.write('\n')

shuffled_rows = [shuffle_options(row) for row in rows]
shuffled_rows = list(filter(lambda row: row is not None, shuffled_rows))
with open(shuffled_path2, 'w') as f:
    for row in shuffled_rows:
        json.dump(row, f, ensure_ascii=False)
        f.write('\n')

In [79]:
raise ""

TypeError: exceptions must derive from BaseException

In [ ]:
import pandas as pd

# Load the Excel file
excel_file_path = "mc.xlsx"
df = pd.read_excel(excel_file_path)

# Save the DataFrame to a CSV file
csv_file_path = "mc.csv"
df.to_csv(csv_file_path, index=False)